In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("project02.ipynb")

<p><img alt="Colaborat" style="height: 220px;" src="https://github.com/CienciaDatosUdea/001_DataSetFundCienciaDatos/blob/main/semestre2024-2/lab/lab02/Escudo-UdeA.svg.png?raw=true" align="left" hspace="10px" vspace="0px"></p>


# Fundamentos en ciencias de datos


# Proyecto 2: Cambio Climático - Temperaturas y Precipitación

¡En este proyecto, investigarás datos sobre el cambio climático, o los cambios a largo plazo en las temperaturas y los patrones climáticos!

### Logística

**Fecha límite.** Este proyecto se debe entregar  el miércoles 26 antes de las **11:00pm**. 

Puedes trabajar con otro compañero

Para comenzar, carga las bibliotecas `datascience`, `numpy` y `matplotlib`. Asegúrate también de ejecutar la primera celda de este cuaderno para cargar `otter`. Estaremos trabajando con varios conjuntos de datos: `city_info.csv`, que contiene información sobre diferentes ciudades; `phoenix.csv`, que presenta datos climáticos como temperatura máxima y mínima, así como precipitación; y `southwest.csv`, que incluye la precipitación total por año en varias ciudades. 

In [ ]:
# Run this cell to set up the notebook, but please don't change it.
from datascience import *
import numpy as np
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
np.set_printoptions(legacy='1.13')

import warnings
warnings.simplefilter('ignore')

## Parte 1: Temperaturas


En el siguiente análisis, investigaremos uno de los problemas más destacados del siglo XXI: **el cambio climático**. Aunque los detalles científicos del clima están fuera del alcance de este curso, podemos comenzar a aprender sobre el cambio climático analizando los registros públicos de temperatura y precipitación en diferentes ciudades a lo largo del tiempo.

Analizaremos una colección de mediciones históricas diarias de temperatura y precipitación de estaciones meteorológicas en 210 ciudades de los EE.UU. El conjunto de datos fue compilado por Yuchuan Lai y David Dzombak [1]. Una descripción de los datos de los autores originales y los datos en sí están disponibles [aquí](https://kilthub.cmu.edu/articles/dataset/Compiled_daily_temperature_and_precipitation_data_for_the_U_S_cities/7890488).

[1] Lai, Yuchuan; Dzombak, David (2019): Conjunto de datos históricos diarios de temperatura y precipitación para 210 ciudades de EE.UU. Universidad Carnegie Mellon. Conjunto de Datos.

### Parte 1, Sección 1: Ciudades


Ejecuta la siguiente celda para cargar la información sobre las `cities` y mostrar una vista previa de las primeras filas.

In [ ]:
cities = Table.read_table('city_info.csv', index_col=0)
cities.show(3)

La tabla `cities` tiene una fila por estación meteorológica y las siguientes columnas:

1. `"Name"`: El nombre de la ciudad de EE.UU.
2. `"ID"`: El identificador único para la ciudad de EE.UU.
3. `"Lat"`: La latitud de la ciudad de EE.UU. (medida en grados de latitud)
4. `"Lon"`: La longitud de la ciudad de EE.UU. (medida en grados de longitud)
5. `"Stn.Name"`: El nombre de la estación meteorológica donde se recopilaron los datos
6. `"Stn.stDate"`: Una cadena que representa la fecha de la primera grabación en esa estación particular
7. `"Stn.edDate"`: Una cadena que representa la fecha de la última grabación en esa estación particular

Los datos enumeran las estaciones meteorológicas en las cuales se recopilaron datos de temperatura y precipitación. Es importante notar que, aunque algunas ciudades tienen múltiples estaciones meteorológicas, solo una está recopilando datos para esa ciudad en un momento dado. Por lo tanto, podemos centrarnos solo en las ciudades en sí mismas.

<!-- BEGIN QUESTION -->

**Pregunta 1.1.1:** En la celda a continuación, realiza un diagrama de dispersión que represente la latitud y longitud de cada ciudad en la tabla `cities`, de manera que las ciudades del norte se ubiquen en la parte superior y las ciudades del oeste a la izquierda.

*Nota*: ¡Está bien representar el mismo punto varias veces!

*Sugerencia*: La latitud es el conjunto de líneas horizontales que mide distancias *al norte o sur* del ecuador. La longitud es el conjunto de líneas verticales que mide distancias *al este o al oeste* del meridiano de Greenwich.

In [ ]:
# x = cities.select('Lat', 'Lon').column(0)
# y = cities.select('Lat', 'Lon').column(1)
# plt.scatter(y,x)

Estas ciudades se encuentran dentro del continente de los Estados Unidos, por lo que la forma general del país debería ser visible en tu gráfico. La forma aparecerá distorsionada en comparación con la mayoría de los mapas por dos razones: el diagrama de dispersión es cuadrado, aunque Estados Unidos es más ancho que alto, y este diagrama de dispersión es una [proyección equirectangular](https://es.wikipedia.org/wiki/Proyecci%C3%B3n_equirectangular) de la Tierra esférica. Un mapa geográfico de los mismos datos utiliza la común [proyección Pseudo-Mercator](https://es.wikipedia.org/wiki/Proyecci%C3%B3n_Web_Mercator).

> *Nota:* Si esta visualización no se carga en tu dispositivo, por favor, visualiza una versión en línea [aquí](https://github.jonathanferrari.com/static/city_map.html). 



In [ ]:
# Just run this cell
Marker.map_table(cities.select('Lat', 'Lon', 'Name').relabeled('Name', 'labels'))

**Pregunta 1.1.3:** Asigna el valor de `num_unique_cities` al número de ciudades únicas que aparecen en la tabla `cities`.

In [ ]:
num_unique_cites = ...

# Do not change this line
print(f"There are {num_unique_cites} unique cities that appear within our dataset.")


In [ ]:
grader.check("q2")

Para investigar más a fondo, será útil determinar en qué región de los Estados Unidos se encuentra cada ciudad: Noreste, Noroeste, Sureste o Suroeste. Para nuestros propósitos, utilizaremos los siguientes límites geográficos:

<img src= "usa_coordinates.png" alt="Mapa de Coordenadas de EE.UU." width="600"/>

1. Una estación se encuentra en la región `"Northeast"` si su latitud es mayor o igual a 40 grados y su longitud es mayor o igual a -100 grados.
2. Una estación se encuentra en la región `"Northwest"` si su latitud es mayor o igual a 40 grados y su longitud es menor de -100 grados.
3. Una estación se encuentra en la región `"Southeast"` si su latitud es menor de 40 grados y su longitud es mayor o igual a -100 grados.
4. Una estación se encuentra en la región `"Southwest"` si su latitud es menor de 40 grados y su longitud es menor de -100 grados.

**Pregunta 1.1.4**: Define la función `coordinates_to_region` a continuación. Debe tomar dos argumentos, la latitud (`lat`) y la longitud (`lon`) de una ciudad, y devolver una cadena que represente la región en la que se encuentra.

In [ ]:
def coordinates_to_region(...):
    ...

In [ ]:
grader.check("q3")

**Pregunta 1.1.5**: Agrega una nueva columna en `cities` llamada `Region` que contenga la región en la cual está ubicada la ciudad. Para obtener la máxima puntuación, debes utilizar la función `coordinates_to_region` que definiste anteriormente en lugar de volver a implementar su lógica. 



In [ ]:
regions_array = ...
cities = ...
cities.show(5)

In [ ]:
grader.check("q4")

Para confirmar que has definido correctamente tu función `coordinates_to_region` y añadido exitosamente la columna `Region` a la tabla `cities`, ejecuta la siguiente celda. Cada región debería tener un color diferente en el resultado.

In [ ]:
# Just run this cell
cities.scatter("Lon", "Lat", group="Region")

**Pregunta Desafío 1.1.6 <span style="color:#BC412B">(OPCIONAL, no calificada)</span>**: Crea una nueva tabla llamada `cities_nearest`. Esta tabla debe contener las mismas columnas que la tabla `cities` y una columna adicional llamada `"Nearest"` que contenga el **nombre de la ciudad más cercana** que se encuentra en una región diferente a la ciudad descrita por la fila.

Para aproximar la distancia entre dos ciudades, calcula la raíz cuadrada de la suma del cuadrado de la diferencia entre sus latitudes y el cuadrado de la diferencia entre sus longitudes. **No uses una declaración `for`; en su lugar, utiliza el método `apply` y la aritmética de arreglos.**

*Consejo*: Hemos definido una función `distancia` para ti, que puede ser llamada con los números `lat0` y `lon0` y los arreglos `lat1` y `lon1`.

In [ ]:

...
cities_nearest = ...
#cities_nearest.show(5)

In [ ]:
cities_nearest

In [ ]:
grader.check("q5")

### Parte 1, Sección 2: Bienvenidos a Phoenix, Arizona

Phoenix es la capital del estado de Arizona y es conocida por su clima cálido y soleado. En esta sección, analizaremos el clima de Phoenix utilizando los datos de los archivos suministrados. Nos enfocaremos en las temperaturas máximas, mínimas y las precipitaciones diarias para comprender mejor las características climáticas de la ciudad a lo largo del tiempo. 

Cada ciudad tiene un archivo CSV diferente lleno de mediciones diarias de temperatura y precipitación. El archivo para Phoenix, Arizona, se incluye en este proyecto como `phoenix.csv`. Los archivos para otras ciudades se pueden descargar [aquí](https://kilthub.cmu.edu/articles/dataset/Compiled_daily_temperature_and_precipitation_data_for_the_U_S_cities/7890488) emparejándolos con el ID de la ciudad en la tabla `ciudades`.

Dado que Phoenix se encuentra en el borde superior del Desierto de Sonora, tiene temperaturas bastante impresionantes.

Ejecuta la siguiente celda para cargar la tabla `phoenix`. Esta tabla tiene una fila por día y las siguientes columnas:

1. `"Date"`: La fecha (una cadena de texto) que representa la fecha del registro en formato **YYYY-MM-DD**
2. `"tmax"`: La temperatura máxima del día (°C)
3. `"tmin"`: La temperatura mínima del día (°C)
4. `"prcp"`: La precipitación registrada para el día (pulgadas)

In [ ]:
phoenix = Table.read_table("phoenix.csv", index_col=0)
phoenix = phoenix.with_columns(
    "tmax", (phoenix.column('tmax') - 32) * (5/9),
    "tmin", (phoenix.column('tmin') - 32) * (5/9)
)
# (Opcional) verificar los primeros registros transformados
phoenix.show(5)

### **Pregunta 1.2.1:**  
Asigna a la variable `largest_2010_range_date` la fecha correspondiente al **mayor rango de temperatura** registrado en Phoenix, Arizona, en cualquier día comprendido entre el **1 de enero de 2010 y el 31 de diciembre de 2010**.  

Para ello, construye una tabla llamada `phoenix_with_ranges_2010`, filtrando la tabla original de Phoenix para incluir únicamente los registros correspondientes a los días del año 2010. Además, agrega una columna adicional que indique el **rango de temperatura** de cada día, calculado como la diferencia entre la temperatura máxima y la mínima.  

**Tu respuesta debe ser una cadena en formato `"AAAA-MM-DD"`** para la variable largest_2010_range_date . Puedes utilizar tantas líneas de código como sea necesario.  

Sugerencia: 
- Para filtrar los valores de una columna que contengan cierto texto, utiliza el predicado `are.` correspondiente de la [Hoja de referencia de Python](https://www.data8.org/fa23/reference/#tablewhere-predicates).  

- No es necesario reasignar la variable `phoenix`; todas las operaciones deben realizarse sobre la tabla `phoenix_with_ranges_2010`.  


In [ ]:
...
phoenix_with_ranges_2010 = ...
largest_2010_range_date = ...
largest_2010_range_date


In [ ]:
grader.check("q6")

Podemos verificar nuestra tabla `phoenix` para revisar las lecturas de temperatura en nuestra fecha de mayor rango en 2010, llamada `largest_2010_range_date`, y así determinar si ocurrió algo inusual. Ejecuta la celda a continuación para encontrar la fila de la tabla `phoenix` que corresponde a la fecha que identificamos anteriormente.

In [ ]:
# Just run this cell
phoenix.where("Date", largest_2010_range_date)

¡VAYA! ¡MIRA la temperatura máxima de ese día! Hace calor.

La función `extract_year_from_date` toma una cadena de texto de fecha en el formato **AAAA-MM-DD** y devuelve un entero que representa el **año**. La función `extract_month_from_date` toma una cadena de texto de fecha y devuelve una cadena que describe el mes. Ejecuta esta celda, no necesitas entender cómo funciona este código ni editarlo.

In [ ]:
# Just run this cell
import calendar

def extract_year_from_date(date):
    """Returns an integer corresponding to the year of the input string's date."""
    return int(date[:4])

def extract_month_from_date(date):
    "Return an abbreviation of the name of the month for a string's date."
    month = date[5:7]
    return f'{month} ({calendar.month_abbr[int(date[5:7])]})'


# Example
print('2022-04-01 tiene el año', extract_year_from_date('2022-04-01'),
      'y el mes', extract_month_from_date('2022-04-01'))

**Pregunta 1.2.2:** Añade dos nuevas columnas llamadas `Year` y `Month` a la tabla `phoenix`. Estas columnas deben contener el año como un **número entero** y el mes como una **cadena de texto** (por ejemplo, `"04 (Abr)"`) para cada día, respectivamente.

*Nota*: ¡Las funciones anteriores pueden ser de ayuda!

In [ ]:
years_array = ...
months_array = ...
...
phoenix.show(5)

In [ ]:
grader.check("q7")

<!-- BEGIN QUESTION -->

**Pregunta 1.2.3:** Usando la tabla `phoenix`, crea un gráfico de líneas superpuesto de la **temperatura máxima promedio** y la **temperatura mínima promedio** para cada año entre 1900 y 2020 (inclusive).



In [ ]:
...
...
...
...
...
plt.xlabel("Year")
plt.ylabel("T(C)")


<!-- BEGIN QUESTION -->

**Pregunta 1.2.4:** Aunque todavía es un tema de intenso debate, muchos científicos del clima coinciden en que los efectos del cambio climático comenzaron a hacerse evidentes a principios de la década de 1960, como resultado de los niveles elevados de emisiones de gases de efecto invernadero. ¿Cómo respalda el gráfico que generaste en la Pregunta 1.2.3 la afirmación de que el calentamiento global moderno comenzó a principios de la década de 1960?

Escribe tu respuesta en esta celda por favor....

Calcular un promedio de temperaturas a lo largo de un año completo puede ocultar algunos efectos del cambio climático. Por ejemplo, si los veranos se vuelven más calurosos pero los inviernos más fríos, el promedio anual podría no variar mucho. Vamos a investigar cómo han cambiado con el tiempo las temperaturas máximas promedio **mensuales** en Phoenix.

**Pregunta 1.2.5:** Crea una tabla llamada `monthly_increases` con una fila por cada mes, y las siguientes cuatro columnas en orden: 
1. `"Month"`: El mes (como `"02 (Feb)"`)
2. `"Past"`: La temperatura máxima promedio en ese mes desde 1900 hasta 1960 (incluidos)
3. `"Present"`: La temperatura máxima promedio en ese mes desde 2019 hasta 2021 (incluidos)
4. `"Increase"`: La diferencia entre las temperaturas máximas promedio del presente y el pasado en ese mes

Primero, haz una copia de la tabla `phoenix` y añade una nueva columna que contenga el **período** correspondiente para cada fila. El período se refiere a si el año está en la categoría de `"Past"`, `"Present"` o `"Other"`. Puede ser útil utilizar la función de período para ver a qué años corresponde cada período. Luego, utiliza esta nueva tabla para construir `monthly_increases`. Siéntete libre de usar tantas líneas como necesites.

*Pista*: ¿Qué método de la tabla podemos usar para obtener cada **valor único** como su propia columna?

*Nota*: Por favor, **no** reasignes la variable `phoenix`.

In [ ]:
def period(year):
    "Output if a year is in the Past, Present, or Other."
    if 1900 <= year <= 1960:
        return "Past"
    elif 2019 <= year <= 2021:
        return "Present"
    else:
        return "Other"
...
monthly_increases = ...
monthly_increases.show()


In [ ]:
grader.check("q10")

### Febrero en Phoenix



Los valores de la columna `"Past"` se promedian a lo largo de muchas décadas, por lo que son estimaciones fiables de las temperaturas máximas promedio en esos meses antes de los efectos del cambio climático moderno. Sin embargo, la columna `"Present"` se basa únicamente en observaciones de tres años. Febrero, el mes más corto, tiene la menor cantidad de observaciones totales: solo 85 días. Ejecuta la celda a continuación para ver esto.

In [ ]:
# Just run this cell
feb_present = phoenix.where('Year', are.between_or_equal_to(2019, 2021)).where('Month', '02 (Feb)')
feb_present.num_rows

Mira nuevamente tu tabla `monthly_increases`. En comparación con los otros meses, el incremento para el mes de febrero es bastante pequeño; la diferencia de febrero es muy cercana a cero. Ejecuta la siguiente celda para imprimir la diferencia observada.

In [ ]:
# Just run this cell
print(f"February Difference: {monthly_increases.row(1).item('Increase')}")

Quizás esa pequeña diferencia se deba al azar. Para investigar esta idea, requiere un experimento mental.

Podemos observar todas las temperaturas máximas de febrero desde 2019 hasta 2021 (el período actual), por lo que tenemos acceso a todo el censo; no hay muestreo aleatorio involucrado. Sin embargo, podemos imaginar que si pasan más años con el mismo clima actual, habría temperaturas máximas diferentes pero similares en futuros días de febrero. A partir de los datos que observamos, podemos intentar estimar la **temperatura máxima promedio de febrero** en esta colección imaginaria de todos los días futuros de febrero que ocurrirían en nuestro clima moderno, suponiendo que el clima no cambie más y pasen muchos años.

También podemos imaginar que la temperatura máxima de cada día es como una **selección aleatoria de una distribución de temperaturas para ese mes**. Tratar las observaciones reales de eventos naturales como si cada una fuera *aleatoriamente* muestreada de alguna distribución desconocida es una suposición simplificada. Estas temperaturas no fueron realmente muestreadas al azar; en cambio, ocurrieron debido a las complejas interacciones del clima de la Tierra, pero tratarlas como si fueran aleatorias abstrae los detalles de este proceso natural y nos permite realizar inferencias estadísticas. Las conclusiones son solo tan válidas como las suposiciones en las que se basan, pero en este caso, pensar en las temperaturas diarias como muestras aleatorias de alguna distribución climática desconocida parece al menos plausible.

Si asumimos que las **temperaturas reales fueron sacadas al azar de una gran población de posibles días de febrero** en nuestro clima moderno, entonces no solo podemos estimar el promedio poblacional de esta distribución, sino también cuantificar nuestra incertidumbre sobre esa estimación utilizando un intervalo de confianza.

**Solo calcularemos el límite inferior de este intervalo de confianza.** Tenemos la intención de comparar nuestro intervalo de confianza con el promedio histórico (es decir, el valor `"Past"` en nuestra tabla `monthly_increases`). En todos los meses, el promedio de la muestra que consideraremos (es decir, el valor `"Present"` en nuestra tabla `monthly_increases`) es _mayor_ que el promedio histórico. Dado que estamos esencialmente interesados en ver si las temperaturas promedio de febrero han ***aumentado*** desde el pasado, solo nos preocupa que el límite inferior del intervalo de confianza sea mayor

**Pregunta 1.2.6.** Completa la implementación de la función `ci_lower`, que toma una tabla de una sola columna `t` que contiene observaciones de muestra y un porcentaje de nivel de confianza `level`, como 95 o 99. Devuelve el límite inferior de un intervalo de confianza para la media poblacional construido utilizando 5.000 remuestreos de bootstrap.

Después de definir `ci_lower`, hemos proporcionado una línea de código que llama a `ci_lower` en las temperaturas máximas de febrero actuales para obtener el límite inferior de un intervalo de confianza del 99% para el promedio de las temperaturas máximas diarias en febrero. El resultado debería estar alrededor de los 19.4 grados.

In [ ]:
def ci_lower(t, level):
    '''Calcula un límite inferior de un intervalo de confianza del level%  
    para el promedio de la población, donde la columna 0 de la Tabla t contiene una muestra.  
    '''
    stats = make_array()
    for k in np.arange(5000):
        stat = ...
        stats = ...
    ...

# Llama a ci_lower sobre las temperaturas máximas de febrero en la actualidad 
# para encontrar el límite inferior de un intervalo de confianza del 99%.
feb_present_ci = ci_lower(feb_present.select('tmax'), 99)
feb_present_ci

In [ ]:
grader.check("q11")

<!-- BEGIN QUESTION -->

**Pregunta 1.2.7** El límite inferior del intervalo de confianza del 99% para `feb_present_ci` está por debajo del promedio observado de la temperatura máxima de febrero en el pasado, que es 20.4713 (según la tabla `monthly_increases`). ¿Qué conclusión puedes sacar sobre el efecto del cambio climático en las temperaturas máximas de febrero en Phoenix a partir de esta información? Utiliza un nivel de significancia del 1% para el valor p.

*Nota*: Si tienes dificultades con esta pregunta, puede ser útil releer los párrafos bajo el encabezado *Febrero* (particularmente los primeros).

Escribe aqui su respuesta...

### Todos los Meses


**Pregunta 1.2.8.** Repite el proceso de comparación entre el **límite inferior de un intervalo de confianza del 99%** y el **promedio pasado** para cada mes. Para cada mes, imprime el nombre del mes (por ejemplo, `02 (Feb)`), el promedio observado en el pasado y el límite inferior de un intervalo de confianza para el promedio actual.

Usa la función `print` proporcionada para formatear el resultado como una línea por mes.

*Consejo*: Tu código debe seguir el mismo formato que nuestro código anterior (es decir, la sección de *febrero*).

In [ ]:
comparisons = make_array()

# Tomamos los meses únicos desde los datos ya filtrados:
months = np.unique(phoenix_periods.column("Month"))

for month in months:
    # Promedio observado en el período 'Past' para el mes actual:
    past_average = ....

    # Todas las observaciones actuales ('Present') para este mes:
    present_observations = ...

    # Límite inferior del intervalo de confianza del 99% sobre las observaciones actuales:
    present_lower_bound = ci_lower(present_observations, 99)

    # NO modificar desde aquí en adelante (código dado por el ejercicio):
    below = past_average < present_lower_bound
    if below:
        comparison = '**abajo**'
    else:
        comparison = '*arriba*'
    comparisons = np.append(comparisons, comparison)

    print('Para', month, 'el promedio pasado', round(past_average, 1), 
      'está', comparison, 
      'del límite inferior', round(present_lower_bound, 1),
      'del intervalo de confianza al 99% del promedio presente. \n')


In [ ]:
print('Para', month, 'el promedio pasado', round(past_average, 1), 
      'está', comparison, 
      'del límite inferior', round(present_lower_bound, 1),
      'del intervalo de confianza al 99% del promedio presente. \n')

In [ ]:
grader.check("q13")

<!-- BEGIN QUESTION -->

**Pregunta 1.2.9.** Resuma sus hallazgos: Después de comparar el promedio histórico con el límite inferior del intervalo de confianza del 99% para cada mes, ¿qué conclusiones podemos hacer sobre la temperatura máxima promedio mensual en los tiempos históricos (1900-1960) frente a los tiempos modernos (2019-2021) en los doce meses? En otras palabras, ¿qué hipótesis nula deberías considerar, y para qué meses rechazarías o no rechazarías la hipótesis nula? Utiliza un punto de corte del valor p del 1%.

*Pista*: ¿Notas algún patrón estacional?

Escribe aqui su respuesta...

--------------------------------------------------

# Parte 2: Sequía


De acuerdo con la [Agencia de Protección Ambiental de los Estados Unidos](https://www.epa.gov/climate-indicators/southwest), "Grandes porciones del Suroeste han experimentado condiciones de sequía desde que comenzaron los registros semanales del Monitor de Sequía en el año 2000. Durante períodos prolongados de 2002 a 2005 y de 2012 a 2020, casi toda la región estuvo anormalmente seca o incluso más seca."

Evaluar el impacto de la sequía es un desafío utilizando solo datos a nivel de ciudad, porque gran parte del agua que utilizan las personas es transportada desde otros lugares. Sin embargo, exploraremos los datos disponibles para ver qué podemos aprender.

Primero, echemos un vistazo a los datos de precipitación en la región del Suroeste. El archivo `southwest.csv` contiene la precipitación total anual para 13 ciudades en el suroeste de los Estados Unidos, registrada para cada año desde 1960 hasta 2021. Este conjunto de datos se ha consolidado a partir de datos diarios e incluye solo las ciudades del Suroeste del conjunto de datos original que tienen registros de precipitación consistentes desde 1960.

In [ ]:
southwest = Table.read_table('southwest.csv')
southwest.show(5)

**Pregunta 2.1.** Crea una tabla llamada `totals` que contenga una fila para cada año en orden cronológico. La tabla debe contener las siguientes columnas:

1. `"Year"`: El año (un número)
2. `"Precipitation"`: La precipitación total en las 13 ciudades del suroeste para ese año

In [ ]:
...
totals = ...

totals.show(5)


In [ ]:
grader.check("q15")

Ejecuta la celda a continuación para graficar la precipitación total en estas ciudades a lo largo del tiempo, de manera que podamos intentar identificar la sequía visualmente. Como recordatorio, los años de sequía proporcionados por la EPA fueron (2002-2005) y (2012-2020).

In [ ]:
# Just run this cell
totals.plot("Year", "Precipitation")

Este gráfico no es muy revelador. Cada año presenta una cantidad diferente de precipitación y hay bastante variabilidad entre los años, como si la precipitación de cada año fuera un resultado aleatorio de una distribución de posibles resultados.

¿Podría ser que estas llamadas "condiciones de sequía" de 2002-2005 y 2012-2020 puedan explicarse por azar? En otras palabras, ¿podría ser que las cantidades anuales de precipitación en el Suroeste durante estos años de sequía son como **resultados aleatorios de la misma distribución subyacente** que en otros años? Tal vez no ha cambiado nada en los patrones de precipitación de la Tierra, y el suroeste de EE.UU. simplemente experimentó algunos años secos muy seguidos.

Para evaluar esta idea, realicemos una prueba A/B en la que **la precipitación total de cada año** es un resultado, y la condición es **si el año está o no en el período de sequía según la EPA**.

Esta función `drought_label` diferencia entre los años de sequía (drought), tal como se describe en la declaración de la EPA de EE. UU. mencionada anteriormente (2002-2005 y 2012-2020), y otros años. Cabe destacar que la etiqueta "Other" puede ser algo engañosa, ya que hubo otras sequías antes del año 2000, como la masiva [sequía de 1988](https://en.wikipedia.org/wiki/1988%E2%80%9390_North_American_drought) que afectó gran parte de EE. UU. Sin embargo, si nos interesa determinar si estos períodos de sequía más recientes (2002-2005 y 2012-2020) son *normales* o *anormales*, tiene sentido diferenciar los años de esta manera.

In [ ]:
def drought_label(n):
    """Return the label for an input year n."""
    if 2002 <= n <= 2005 or 2012 <= n <= 2020:
        return 'drought'
    else:
        return 'other'

<!-- BEGIN QUESTION -->

**Pregunta 2.2.** Define las hipótesis nula y alternativa para un test A/B que investiga si los años de sequía son **más secos** (tienen menos precipitación) que otros años.

*Nota:* Por favor, formatea tu respuesta utilizando la siguiente estructura.

- *Hipótesis nula:* ...
- *Hipótesis alternativa:* ...



In [ ]:
hipotesis nula=" ... "

hipotesis_alternativa="... "


In [ ]:
print('Hipótesis nula: Los años de sequía no son más secos que otros años; es decir, la precipitación media en años de sequía es igual a la precipitación media en otros años.')
print('Hipótesis alternativa: Los años de sequía son más secos que otros años; es decir, la precipitación media en años de sequía es menor que la precipitación media en otros años.')

**Pregunta 2.3.** Primero, define la tabla `drought`. Debe contener una fila por año y las siguientes dos columnas:

- `"Label"`: Indica si un año es parte de un año de `"drought"` o de un año `"other"`
- `"Precipitation"`: La suma de la precipitación total en 13 ciudades del suroeste en ese año

Luego, construye un histograma superpuesto de dos distribuciones observadas: la precipitación total en años de sequía y la precipitación total en otros años.

*Nota*: Usa los `bins` proporcionados al crear tu histograma, y no reasignes la tabla `southwest`. ¡Siéntete libre de usar tantas líneas como necesites!

*Sugerencia*: ¡El argumento opcional `group` en cierta función podría ser útil!

In [ ]:

drought = ...
bins = np.arange(85, 215+1, 13)
drought.hist('Precipitation', group='Label', bins=bins)


In [ ]:
grader.check("q17")

Antes de continuar, examina el histograma que acabas de crear e intenta adivinar la conclusión de la prueba A/B. Desarrollar la intuición sobre el resultado de las pruebas de hipótesis a partir de visualizaciones es muy útil para aplicaciones en ciencia de datos.

**Pregunta 2.4.** Nuestro próximo paso es elegir una estadística de prueba basada en nuestra hipótesis alternativa de la Pregunta 2.2. ¿Cuál de las siguientes opciones son elecciones válidas para la estadística de prueba? Asigna `ab_test_stat` a **un arreglo de enteros** que corresponda a las opciones válidas. Supón que se consideran promedios y totales a partir de las sumas de precipitación total para cada año.

1. La diferencia entre la precipitación **total** en los años de **sequía** y la precipitación **total** en los **otros** años.
2. La diferencia entre la precipitación **total** en los **otros** años y la precipitación **total** en los años de **sequía**.
3. La diferencia **absoluta** entre la precipitación **total** en los otros años y la precipitación **total** en los años de sequía.
4. La diferencia entre la precipitación **promedio** en los años de **sequía** y la precipitación **promedio** en los **otros** años.
5. La diferencia entre la precipitación **promedio** en los **otros** años y la precipitación **promedio** en los años de **sequía**.
6. La diferencia **absoluta** entre la precipitación **promedio** en los otros años y la precipitación **promedio** en los años de sequía.

In [ ]:
ab_test_stat = make_array(...)


In [ ]:
grader.check("q18")

<!-- BEGIN QUESTION -->

**Pregunta 2.5.** Los científicos del clima, Noah y Sarah, señalan que hay más años **normales** que años de **sequía**, por lo que medir la diferencia en la precipitación total siempre favorecerá a los años **normales**. Ellos concluyen que todas las opciones anteriores que implican la precipitación **total** son elecciones inválidas para una estadística de prueba. ¿Estás de acuerdo con ellos? ¿Por qué sí o por qué no?

> *Pista:* ¡Piensa en cómo funcionan las pruebas de permutación con clases desequilibradas!

Escribe tu respuesta aqui...

Antes de continuar, revisa tu tabla `drought`. Debe tener dos columnas, `labels` y `Precipitation`, con 61 filas, de las cuales 13 corresponden a años de "sequía"(drought).

In [ ]:
drought.show(3)

In [ ]:
drought.group('Label')

**Pregunta 2.6.** Para nuestra prueba A/B, usaremos la diferencia entre la precipitación promedio en los años de sequía y la precipitación promedio en otros años como nuestra estadística de prueba:

$$\text{precipitación promedio en años de ``sequía''} - \text{precipitación promedio en ``otros'' años}$$

Primero, completa la función `test_statistic`. Debe recibir una tabla de dos columnas `t` con una fila por año y dos columnas:
- `Label`: la etiqueta para ese año (puede ser `'drought'` o `'other'`)
- `Precipitation`: la precipitación total en las 13 ciudades del Suroeste ese año. 

Luego, usa la función que defines para asignar `observed_statistic` a la estadística de prueba observada.

El archivo que se utiliza para la regresión es faithful.csv, que tiene las columnas `duration` y `wait`.

In [ ]:
def test_statistic(t):
    ...

observed_statistic = ...
observed_statistic


In [ ]:
grader.check("q20")

Ahora que hemos definido nuestras hipótesis y el estadístico de prueba, estamos listos para llevar a cabo nuestra prueba de hipótesis. Comenzaremos definiendo una función para simular el estadístico de prueba bajo la hipótesis nula y luego llamaremos a esa función 5000 veces para construir una distribución empírica bajo la hipótesis nula.

**Pregunta 2.7.** Escribe una función para simular el estadístico de prueba bajo la hipótesis nula. La función `simulate_precipitation_null` debe simular la hipótesis nula una vez (no 5,000 veces) y devolver el valor del estadístico de prueba para esa muestra simulada.

*Sugerencia*: Usar `t.with_column(...)` con un nombre de columna que ya existe en una tabla `t` reemplazará esa columna con los valores especificados recientemente.

In [ ]:
def simulate_precipitation_null():
    ...

# Run your function a couple times to make sure that it works
simulate_precipitation_null()


In [ ]:
plt.hist(np.array([simulate_precipitation_null()  for i in range (0, 10000)]))

In [ ]:
grader.check("q21")

**Pregunta 2.8.** Completa los espacios en blanco a continuación para finalizar la simulación de la prueba de hipótesis. Tu simulación debe calcular 5,000 valores del estadístico de prueba bajo la hipótesis nula y almacenar el resultado en el arreglo `sampled_stats`.

*Consejo:* Deberías usar la función `simulate_precipitation_null` que escribiste en la pregunta anterior.

*Nota:* Ejecutar esta celda puede tomar unos segundos. Si tarda más de un minuto, intenta encontrar una manera diferente (más rápida) de implementar tu función `simulate_precipitation_null`.

In [ ]:
repetitions = 5000
sampled_stats = make_array()

for i in np.arange(repetitions):
    sampled_stats = np.append(... simulate_precipitation_null())

# Do not change these lines
Table().with_column('Difference Between Means', sampled_stats).hist()
plt.scatter(observed_statistic, 0, c="r", s=50);
plt.ylim(-0.01);


In [ ]:
grader.check("q22")

**Pregunta 2.9.** Calcula el valor p para esta prueba de hipótesis y asígnalo a la variable `precipitation_p_val`.

In [ ]:
precipitation_p_val =...
precipitation_p_val

In [ ]:
grader.check("q23")

<!-- BEGIN QUESTION -->

**Pregunta 2.10.** Establece una conclusión a partir de esta prueba utilizando un valor p de referencia del 5%. ¿Qué has aprendido sobre la declaración de la EPA con respecto a la sequía?

Escriba su respuesta aqui...

**Pregunta 2.11.** ¿Se aplica tu conclusión de la Pregunta 2.10 a toda la región del suroeste de los EE. UU.? ¿Por qué o por qué no?

> *Nota:* ¡Siéntete libre de investigar sobre las características geográficas de esta región de los EE. UU.!

Escriba su respuesta aqui...

<!-- FIN DE LA PREGUNTA -->

# Conclusión


La ciencia de datos juega un papel central en la investigación sobre el cambio climático, ya que es necesario realizar simulaciones masivas del clima terrestre para evaluar las implicaciones de los datos climáticos registrados por estaciones meteorológicas, satélites y otros sensores. [Berkeley Earth](http://berkeleyearth.org/data/) es una fuente común de datos para este tipo de proyectos.

En este proyecto, encontramos formas de aplicar nuestras técnicas de inferencia estadística que se basan en el muestreo aleatorio, incluso en situaciones en las que los datos no fueron generados aleatoriamente, sino por algún proceso natural complicado que parecía aleatorio. Hicimos suposiciones sobre la aleatoriedad y luego llegamos a conclusiones basadas en esas suposiciones. Se debe tener mucho cuidado al elegir supuestos que sean realistas, para que las conclusiones resultantes no sean engañosas. No obstante, hacer suposiciones sobre los datos puede ser útil cuando hacerlo permite aplicar técnicas de inferencia a situaciones novedosas.

<img src="Paco.jpeg" alt="Foto de un gato tuxedo descansando en la cama" width="350"/>
Nuestro buen amigo Paco estuvo atento a todo tu desarrollo y quiere Felicitarte  por finalizar este proyecto.
¡Felicitaciones!

***Recordatorios Importantes:***
- Si trabajaste con un conmpanero, cada uno debe subir el archivo ipynb a los enlaces del curso

## Envío

Asegúrate de haber ejecutado todas las celdas de tu cuaderno en orden antes de correr la celda de abajo, para que todas las imágenes/gráficas aparezcan en el resultado. La celda a continuación generará un archivo zip para que lo puedas enviar. **¡Por favor guarda antes de exportar!**

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

These are some submission instructions.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)